In [1]:
import sqlite3
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
import datetime as dt

from sqlite3 import Error
from keras import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from keras.layers import LeakyReLU
from keras.callbacks import CSVLogger
import matplotlib.pyplot as plt
import math



kaggle_data = sqlite3.connect('E:/database/database.sqlite')

In [2]:
# Reading the Kaggle dataset

Country = pd.read_sql_query("SELECT * FROM Country", kaggle_data)
Match_raw = pd.read_sql_query("SELECT * FROM Match", kaggle_data)
Player_Attributes = pd.read_sql_query("SELECT * FROM Player_Attributes", kaggle_data)

In [3]:
# Dropping unnecessary columns in the match dataset

betting_odds_indices = np.arange(Match_raw.columns.get_loc('BWH'),Match_raw.columns.get_loc('BSA')+1,1)
Match_raw = Match_raw.drop(Match_raw.columns[betting_odds_indices], axis=1)
X_Y_player_indices = np.arange(Match_raw.columns.get_loc("home_player_X1"),Match_raw.columns.get_loc("away_player_Y11")+1,1)
Match = Match_raw.drop(Match_raw.columns[X_Y_player_indices], axis=1) 
Match = Match.drop(['id','league_id','stage','shoton','shotoff','foulcommit','card','cross','corner','possession','goal'], axis=1) 
Match = Match.dropna(subset= ['B365H','B365D','B365A'])

# Dropping matches from smaller leagues (none - big 5 leagues)
Match = Match.drop(Match[Match['country_id']==1].index)
Match = Match.drop(Match[Match['country_id']==13274].index)
Match = Match.drop(Match[Match['country_id']==15722].index)
Match = Match.drop(Match[Match['country_id']==17642].index)
Match = Match.drop(Match[Match['country_id']==19694].index)
Match = Match.drop(Match[Match['country_id']==24558].index)

In [4]:
# Reading the match-level elo dataset and merging

filepath = 'E:/database/Elos.csv'
elos = pd.read_csv(filepath)
elos_keep=elos[['match_api_id','Home_Elo','Away_Elo']]

In [5]:
Match  = Match.merge(elos_keep, how = 'left',left_on='match_api_id',right_on = 'match_api_id')

In [6]:
# Creating year and half variables, player attributes will be merged to matches based on year-half

Match['year'] = pd.to_datetime(Match['date']).dt.year
Match['month'] = pd.to_datetime(Match['date']).dt.month
Player_Attributes['year'] = pd.to_datetime(Player_Attributes['date']).dt.year
Player_Attributes['month'] = pd.to_datetime(Player_Attributes['date']).dt.month
Match['half'] = np.where(Match['month'] <7 , 1, 2)
Player_Attributes['half'] = np.where(Player_Attributes['month'] <7 , 1, 2)

# Sorting all datasets based on dates
Match = Match.sort_values(by='date',ascending = True)
Player_Attributes = Player_Attributes.sort_values(by='date',ascending = True)

In [7]:
# Creating labels for match results
# 1 is home team win, -1 is home team loss, 0 is draw

Match['Result'] = Match['home_team_goal'] - Match['away_team_goal']
Match['Result'].loc[Match.Result > 0 ] = 1
Match['Result'].loc[Match.Result < 0] = -1

In [8]:
#Team form feature creation
#Match Form is based on previous 5 home/away match results

def Home_Team_Match_Form (df):
        
    team = df['home_team_api_id']
    date = df['date']
    team_matches = Match[(Match['home_team_api_id'] == team)]
    last_matches = team_matches[team_matches['date'] < date ].sort_values(by = 'date', ascending = False).iloc[0:5,:]
    if last_matches['match_api_id'].count()>=5:
        score = last_matches['Result'].sum()
    elif last_matches['match_api_id'].count()<5:
        score = -100
    return score

def Away_Team_Match_Form (df):
    
    team = df['away_team_api_id']
    date = df['date']
    team_matches = Match[(Match['away_team_api_id'] == team)]
    last_matches = team_matches[team_matches['date'] < date ].sort_values(by = 'date', ascending = False).iloc[0:5,:]
    if last_matches['match_api_id'].count()>=5:
        score = last_matches['Result'].sum()
    elif last_matches['match_api_id'].count()<5:
        score = -100
    return score

In [9]:
Match['Home_Team_Match_Form'] = Match.apply(lambda x: Home_Team_Match_Form(x),axis=1 )
Match['Away_Team_Match_Form'] = Match.apply(lambda x: Away_Team_Match_Form(x),axis=1 )

In [10]:
#Team Offensive form feature creation
#Attacking Form is based on goals scored in previous 5 home/away matches

def Home_Team_Offensive_Form (df):
        
    team = df['home_team_api_id']
    date = df['date']
    team_matches = Match[(Match['home_team_api_id'] == team)]
    last_matches = team_matches[team_matches['date'] < date ].sort_values(by = 'date', ascending = False).iloc[0:5,:]
    if last_matches['match_api_id'].count()>=5:
        score = last_matches['home_team_goal'].sum()
    elif last_matches['match_api_id'].count()<5:
        score = -100
    return score

def Away_Team_Offensive_Form (df):
    
    team = df['away_team_api_id']
    date = df['date']
    team_matches = Match[(Match['away_team_api_id'] == team)]
    last_matches = team_matches[team_matches['date'] < date ].sort_values(by = 'date', ascending = False).iloc[0:5,:]
    if last_matches['match_api_id'].count()>=5:
        score = last_matches['away_team_goal'].sum()
    elif last_matches['match_api_id'].count()<5:
        score = -100
    return score

In [11]:
Match['Home_Team_Offensive_Form'] = Match.apply(lambda x: Home_Team_Offensive_Form(x),axis=1 )
Match['Away_Team_Offensive_Form'] = Match.apply(lambda x: Away_Team_Offensive_Form(x),axis=1 )

In [12]:
#Team Defensive form feature creation
#Defensive Form is based on goals conceded in previous 5 home/away matches

def Home_Team_Defensive_Form (df):
        
    team = df['home_team_api_id']
    date = df['date']
    team_matches = Match[(Match['home_team_api_id'] == team)]
    last_matches = team_matches[team_matches['date'] < date ].sort_values(by = 'date', ascending = False).iloc[0:5,:]
    if last_matches['match_api_id'].count()>=5:
        score = last_matches['away_team_goal'].sum()
    elif last_matches['match_api_id'].count()<5:
        score = -100
    return score

def Away_Team_Defensive_Form (df):
    
    team = df['away_team_api_id']
    date = df['date']
    team_matches = Match[(Match['away_team_api_id'] == team)]
    last_matches = team_matches[team_matches['date'] < date ].sort_values(by = 'date', ascending = False).iloc[0:5,:]
    if last_matches['match_api_id'].count()>=5:
        score = last_matches['home_team_goal'].sum()
    elif last_matches['match_api_id'].count()<5:
        score = -100
    return score

In [13]:
Match['Home_Team_Defensive_Form'] = Match.apply(lambda x: Home_Team_Defensive_Form(x),axis=1 )
Match['Away_Team_Defensive_Form'] = Match.apply(lambda x: Away_Team_Defensive_Form(x),axis=1 )

In [14]:
#Dropping matches with insufficient form features
Match = Match[ (Match['Home_Team_Match_Form'] != -100) & (Match['Home_Team_Offensive_Form'] != -100) & (Match['Home_Team_Defensive_Form'] != -100) & (Match['Away_Team_Match_Form'] != -100) & (Match['Away_Team_Offensive_Form'] != -100) & (Match['Away_Team_Offensive_Form'] != -100)]

In [15]:
# Dropping matches that do not have player data
Match2 = Match.dropna(subset= ['home_player_1','home_player_2','home_player_3','home_player_4','home_player_5',\
                           'home_player_6','home_player_7','home_player_8','home_player_9','home_player_10',\
                           'home_player_11','away_player_1','away_player_2','away_player_3','away_player_4',
                           'away_player_5','away_player_6','away_player_7','away_player_8','away_player_9',
                           'away_player_10','away_player_11'])

In [16]:
# Deleting datapoints that lack identifying information, duplicates, and unnecessary columns in the player attributes dataset

Player_Attributes = Player_Attributes.dropna(subset= ['player_api_id','year','half'])
Player_Attributes = Player_Attributes.drop_duplicates(subset=['player_api_id','year','half'])
Player_Attributes = Player_Attributes.drop(['id','player_fifa_api_id','potential','preferred_foot','attacking_work_rate','defensive_work_rate','month','date'],axis=1)

In [17]:
player_h1 = Player_Attributes.copy()
player_h1.columns = ['player_api_id','overall_rating_h1','crossing_h1','finishing_h1','heading_accuracy_h1','short_passing_h1','volleys_h1','dribbling_h1','curve_h1','free_kick_accuracy_h1','long_passing_h1','ball_control_h1','acceleration_h1','sprint_speed_h1','agility_h1','reactions_h1','balance_h1','shot_power_h1','jumping_h1','stamina_h1','strength_h1','long_shots_h1','aggression_h1','interceptions_h1','positioning_h1','vision_h1','penalties_h1','marking_h1','standing_tackle_h1','sliding_tackle_h1','gk_diving_h1','gk_handling_h1','gk_kicking_h1','gk_positioning_h1','gk_reflexes_h1','year','half']
player_h1.to_csv('E:/database/player_h1.csv')
# Match2  = Match2.merge(player_h1, how = 'left',left_on=['home_player_1','year','half'],right_on = ['player_api_id','year','half'])

In [18]:
player_a1 = Player_Attributes.copy()
player_a1.columns = ['player_api_id','overall_rating_a1','crossing_a1','finishing_a1','heading_accuracy_a1','short_passing_a1','volleys_a1','dribbling_a1','curve_a1','free_kick_accuracy_a1','long_passing_a1','ball_control_a1','acceleration_a1','sprint_speed_a1','agility_a1','reactions_a1','balance_a1','shot_power_a1','jumping_a1','stamina_a1','strength_a1','long_shots_a1','aggression_a1','interceptions_a1','positioning_a1','vision_a1','penalties_a1','marking_a1','standing_tackle_a1','sliding_tackle_a1','gk_diving_a1','gk_handling_a1','gk_kicking_a1','gk_positioning_a1','gk_reflexes_a1','year','half']
player_a1.to_csv('E:/database/player_a1.csv')
# Match2  = Match2.merge(player_a1, how = 'left',left_on=['away_player_1','year','half'],right_on = ['player_api_id','year','half'])

In [19]:
Match2.to_csv('E:/database/match.csv')

In [20]:
# Ran into an error when trying to merge on Python, so used another language (SAS) to merge the data
# The SAS code is uploaded on github
# Home wins have been relabelled as 2, draws as 1, and home losses as 0
# Averaged player attributes for each team (34 attributes per player, 11 players per team, 2 teams per match)
# 12,633 datapoints in total, each datapoint has 11 team level features, 68 player-level features (34 for each team)

match = pd.read_csv('E:/database/merged.csv')

In [21]:
train = match[(match['season']=='2008/2009') | (match['season']=='2009/2010') | (match['season']=='2010/2011') | (match['season']=='2011/2012') | (match['season']=='2012/2013') | (match['season']=='2013/2014')]

dev = match[(match['season']=='2014/2015')]

test = match[(match['season']=='2015/2016')]

In [22]:
train = train.drop(['season'], axis=1)
dev = dev.drop(['season'], axis=1)
test = test.drop(['season'], axis=1)

In [23]:
train_features = train.drop(['result'],axis=1)
train_labels = train[['match_api_id','result']]
dev_features = dev.drop(['result'],axis=1)
dev_labels = dev[['match_api_id','result']]
test_features = test.drop(['result'],axis=1)
test_labels = test[['match_api_id','result']]

In [24]:
train_features.to_csv('E:/database/data/train_features.csv')
train_labels.to_csv('E:/database/data/train_labels.csv')
dev_features.to_csv('E:/database/data/dev_features.csv')
dev_labels.to_csv('E:/database/data/dev_labels.csv')
test_features.to_csv('E:/database/data/test_features.csv')
test_labels.to_csv('E:/database/data/test_labels.csv')
match.to_csv('E:/database/data/combined.csv')

In [25]:
X_train = pd.read_csv('E:/database/data/train_features.csv')
y_train = pd.read_csv('E:/database/data/train_labels.csv')
X_dev = pd.read_csv('E:/database/data/dev_features.csv')
y_dev = pd.read_csv('E:/database/data/dev_labels.csv')
X_test = pd.read_csv('E:/database/data/test_features.csv')
y_test = pd.read_csv('E:/database/data/test_labels.csv')

In [26]:
X_train['H_odds']=1/X_train['B365H']
X_train['D_odds']=1/X_train['B365D']
X_train['A_odds']=1/X_train['B365A']

X_test['H_odds']=1/X_test['B365H']
X_test['D_odds']=1/X_test['B365D']
X_test['A_odds']=1/X_test['B365A']

X_dev['H_odds']=1/X_dev['B365H']
X_dev['D_odds']=1/X_dev['B365D']
X_dev['A_odds']=1/X_dev['B365A']

In [27]:
X_train = X_train.drop(['Unnamed: 0','match_api_id','B365H','B365D','B365A'],axis=1)
y_train = y_train.drop(['Unnamed: 0','match_api_id'],axis=1)
X_dev = X_dev.drop(['Unnamed: 0','match_api_id','B365H','B365D','B365A'],axis=1)
y_dev = y_dev.drop(['Unnamed: 0','match_api_id'],axis=1)
X_test = X_test.drop(['Unnamed: 0','match_api_id','B365H','B365D','B365A'],axis=1)
y_test = y_test.drop(['Unnamed: 0','match_api_id'],axis=1)

In [31]:
X_train=X_train.to_numpy()
X_dev=X_dev.to_numpy()
X_test=X_test.to_numpy()
y_train=pd.get_dummies(y_train).to_numpy()
y_dev=pd.get_dummies(y_dev).to_numpy()
y_test=pd.get_dummies(y_test).to_numpy()

In [32]:
def one_hot(y, c):
    
    y = np.transpose(y)
    y_hot = np.zeros((y.size, y.max()+1), dtype=int)
    y_hot[np.arange(y.size),y] = 1 
    return y_hot

In [33]:
Y_train=one_hot(y_train,3)
Y_test=one_hot(y_test,3)
Y_dev=one_hot(y_dev,3)

In [34]:
def train_model(features, labels, learning_rate, alpha_leakyrelu, dropout, bsize, numepochs):

    """
    5 hidden layers with 64 neurons each
    """    
    
    model = Sequential()
    model.add(Dense(64,input_dim=features.shape[1], kernel_initializer='normal'))
    model.add(LeakyReLU(alpha=alpha_leakyrelu))
    model.add(Dropout(dropout))
    model.add(Dense(64,kernel_initializer='normal'))
    model.add(LeakyReLU(alpha=alpha_leakyrelu))
    model.add(Dropout(dropout))
    model.add(Dense(64,kernel_initializer='normal'))
    model.add(LeakyReLU(alpha=alpha_leakyrelu))
    model.add(Dropout(dropout))
    model.add(Dense(64,kernel_initializer='normal'))
    model.add(LeakyReLU(alpha=alpha_leakyrelu))
    model.add(Dropout(dropout))
    model.add(Dense(64,kernel_initializer='normal'))
    model.add(LeakyReLU(alpha=alpha_leakyrelu))
    model.add(Dense(labels.shape[1], kernel_initializer='normal', activation='softmax'))
    opt = Adam(lr=learning_rate)
    
    
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

    model.summary()
    
    
    
    csv_logger = CSVLogger('log.csv', append=True, separator=';')
    trained = model.fit(features,labels,epochs=numepochs,batch_size=bsize, callbacks=[csv_logger])
    loss_history =  trained.history["loss"]
    accuracy_history =  trained.history["accuracy"]
    
    return model, loss_history, accuracy_history

In [35]:
model, loss_history, accuracy_history = train_model(X_train, Y_train, 0.00001, 0.1, 0.20, 256, 1000)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                5120      
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 64)                0         
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 64)                4160      
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 64)                0         
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 64)                4

37/37 [==============================] - 0s 8ms/step - loss: 1.0548 - accuracy: 0.4690
Epoch 59/1000
37/37 [==============================] - 0s 7ms/step - loss: 1.0545 - accuracy: 0.4659
Epoch 60/1000
37/37 [==============================] - 0s 7ms/step - loss: 1.0561 - accuracy: 0.4671
Epoch 61/1000
37/37 [==============================] - 0s 7ms/step - loss: 1.0560 - accuracy: 0.4678
Epoch 62/1000
37/37 [==============================] - 0s 8ms/step - loss: 1.0572 - accuracy: 0.4642
Epoch 63/1000
37/37 [==============================] - 0s 8ms/step - loss: 1.0555 - accuracy: 0.4700
Epoch 64/1000
37/37 [==============================] - 0s 7ms/step - loss: 1.0530 - accuracy: 0.4669
Epoch 65/1000
37/37 [==============================] - 0s 7ms/step - loss: 1.0550 - accuracy: 0.4684
Epoch 66/1000
37/37 [==============================] - 0s 7ms/step - loss: 1.0545 - accuracy: 0.4682
Epoch 67/1000
37/37 [==============================] - 0s 7ms/step - loss: 1.0522 - accuracy: 0.4692
Epoc

37/37 [==============================] - 0s 7ms/step - loss: 1.0183 - accuracy: 0.5063
Epoch 139/1000
37/37 [==============================] - 0s 7ms/step - loss: 1.0172 - accuracy: 0.5017
Epoch 140/1000
37/37 [==============================] - 0s 7ms/step - loss: 1.0151 - accuracy: 0.4973
Epoch 141/1000
37/37 [==============================] - 0s 6ms/step - loss: 1.0173 - accuracy: 0.4996
Epoch 142/1000
37/37 [==============================] - 0s 7ms/step - loss: 1.0155 - accuracy: 0.5025
Epoch 143/1000
37/37 [==============================] - 0s 7ms/step - loss: 1.0172 - accuracy: 0.5003
Epoch 144/1000
37/37 [==============================] - 0s 6ms/step - loss: 1.0118 - accuracy: 0.5070
Epoch 145/1000
37/37 [==============================] - 0s 6ms/step - loss: 1.0136 - accuracy: 0.5021
Epoch 146/1000
37/37 [==============================] - 0s 7ms/step - loss: 1.0111 - accuracy: 0.5068
Epoch 147/1000
37/37 [==============================] - 0s 7ms/step - loss: 1.0163 - accuracy: 0.

37/37 [==============================] - 0s 6ms/step - loss: 1.0028 - accuracy: 0.5129
Epoch 219/1000
37/37 [==============================] - 0s 6ms/step - loss: 1.0035 - accuracy: 0.5123
Epoch 220/1000
37/37 [==============================] - 0s 6ms/step - loss: 1.0028 - accuracy: 0.5116
Epoch 221/1000
37/37 [==============================] - 0s 6ms/step - loss: 1.0007 - accuracy: 0.5098
Epoch 222/1000
37/37 [==============================] - 0s 7ms/step - loss: 1.0038 - accuracy: 0.5157
Epoch 223/1000
37/37 [==============================] - 0s 7ms/step - loss: 1.0043 - accuracy: 0.5086
Epoch 224/1000
37/37 [==============================] - 0s 7ms/step - loss: 1.0025 - accuracy: 0.5128
Epoch 225/1000
37/37 [==============================] - 0s 7ms/step - loss: 0.9999 - accuracy: 0.5143
Epoch 226/1000
37/37 [==============================] - 0s 6ms/step - loss: 1.0057 - accuracy: 0.5106
Epoch 227/1000
37/37 [==============================] - 0s 6ms/step - loss: 1.0030 - accuracy: 0.

37/37 [==============================] - 0s 6ms/step - loss: 0.9997 - accuracy: 0.5139
Epoch 299/1000
37/37 [==============================] - 0s 7ms/step - loss: 0.9997 - accuracy: 0.5189
Epoch 300/1000
37/37 [==============================] - 0s 7ms/step - loss: 0.9994 - accuracy: 0.5118
Epoch 301/1000
37/37 [==============================] - 0s 7ms/step - loss: 0.9993 - accuracy: 0.5156
Epoch 302/1000
37/37 [==============================] - 0s 8ms/step - loss: 0.9997 - accuracy: 0.5188
Epoch 303/1000
37/37 [==============================] - 0s 6ms/step - loss: 1.0010 - accuracy: 0.5182
Epoch 304/1000
37/37 [==============================] - 0s 6ms/step - loss: 0.9992 - accuracy: 0.5171
Epoch 305/1000
37/37 [==============================] - 0s 6ms/step - loss: 0.9982 - accuracy: 0.5161
Epoch 306/1000
37/37 [==============================] - 0s 6ms/step - loss: 0.9983 - accuracy: 0.5146
Epoch 307/1000
37/37 [==============================] - 0s 7ms/step - loss: 1.0000 - accuracy: 0.

...

37/37 [==============================] - 0s 10ms/step - loss: 0.9937 - accuracy: 0.5181
Epoch 619/1000
37/37 [==============================] - 0s 8ms/step - loss: 0.9930 - accuracy: 0.5197
Epoch 620/1000
37/37 [==============================] - 0s 8ms/step - loss: 0.9936 - accuracy: 0.5190
Epoch 621/1000
37/37 [==============================] - 0s 9ms/step - loss: 0.9923 - accuracy: 0.5197
Epoch 622/1000
37/37 [==============================] - 0s 8ms/step - loss: 0.9912 - accuracy: 0.5206
Epoch 623/1000
37/37 [==============================] - 0s 8ms/step - loss: 0.9925 - accuracy: 0.5207
Epoch 624/1000
37/37 [==============================] - 0s 7ms/step - loss: 0.9918 - accuracy: 0.5187
Epoch 625/1000
37/37 [==============================] - 0s 7ms/step - loss: 0.9939 - accuracy: 0.5198
Epoch 626/1000
37/37 [==============================] - 0s 7ms/step - loss: 0.9919 - accuracy: 0.5201
Epoch 627/1000
37/37 [==============================] - 0s 7ms/step - loss: 0.9936 - accuracy: 0

37/37 [==============================] - 0s 12ms/step - loss: 0.9915 - accuracy: 0.5200
Epoch 699/1000
37/37 [==============================] - 0s 12ms/step - loss: 0.9905 - accuracy: 0.5200
Epoch 700/1000
37/37 [==============================] - 0s 10ms/step - loss: 0.9909 - accuracy: 0.5211
Epoch 701/1000
37/37 [==============================] - 0s 10ms/step - loss: 0.9906 - accuracy: 0.5214
Epoch 702/1000
37/37 [==============================] - 0s 12ms/step - loss: 0.9916 - accuracy: 0.5209
Epoch 703/1000
37/37 [==============================] - 0s 9ms/step - loss: 0.9901 - accuracy: 0.5209
Epoch 704/1000
37/37 [==============================] - 1s 15ms/step - loss: 0.9914 - accuracy: 0.5186
Epoch 705/1000
37/37 [==============================] - 0s 9ms/step - loss: 0.9921 - accuracy: 0.5197
Epoch 706/1000
37/37 [==============================] - 0s 8ms/step - loss: 0.9917 - accuracy: 0.5211
Epoch 707/1000
37/37 [==============================] - 0s 13ms/step - loss: 0.9909 - accur

37/37 [==============================] - 0s 11ms/step - loss: 0.9916 - accuracy: 0.5213
Epoch 779/1000
37/37 [==============================] - 0s 10ms/step - loss: 0.9906 - accuracy: 0.5216
Epoch 780/1000
37/37 [==============================] - 0s 11ms/step - loss: 0.9901 - accuracy: 0.5217
Epoch 781/1000
37/37 [==============================] - 0s 7ms/step - loss: 0.9900 - accuracy: 0.5226
Epoch 782/1000
37/37 [==============================] - 0s 7ms/step - loss: 0.9902 - accuracy: 0.5211
Epoch 783/1000
37/37 [==============================] - 0s 7ms/step - loss: 0.9919 - accuracy: 0.5196
Epoch 784/1000
37/37 [==============================] - 0s 8ms/step - loss: 0.9901 - accuracy: 0.5202
Epoch 785/1000
37/37 [==============================] - 0s 10ms/step - loss: 0.9903 - accuracy: 0.5217
Epoch 786/1000
37/37 [==============================] - 0s 13ms/step - loss: 0.9918 - accuracy: 0.5201
Epoch 787/1000
37/37 [==============================] - 0s 7ms/step - loss: 0.9909 - accurac

37/37 [==============================] - 0s 9ms/step - loss: 0.9904 - accuracy: 0.5219
Epoch 859/1000
37/37 [==============================] - 0s 9ms/step - loss: 0.9908 - accuracy: 0.5212
Epoch 860/1000
37/37 [==============================] - 0s 9ms/step - loss: 0.9896 - accuracy: 0.5220
Epoch 861/1000
37/37 [==============================] - 0s 10ms/step - loss: 0.9909 - accuracy: 0.5222
Epoch 862/1000
37/37 [==============================] - 0s 7ms/step - loss: 0.9894 - accuracy: 0.5228
Epoch 863/1000
37/37 [==============================] - 0s 7ms/step - loss: 0.9893 - accuracy: 0.5234
Epoch 864/1000
37/37 [==============================] - 0s 8ms/step - loss: 0.9907 - accuracy: 0.5230
Epoch 865/1000
37/37 [==============================] - 0s 7ms/step - loss: 0.9906 - accuracy: 0.5209
Epoch 866/1000
37/37 [==============================] - 0s 7ms/step - loss: 0.9925 - accuracy: 0.5204
Epoch 867/1000
37/37 [==============================] - 0s 11ms/step - loss: 0.9904 - accuracy: 

37/37 [==============================] - 0s 6ms/step - loss: 0.9902 - accuracy: 0.5232
Epoch 939/1000
37/37 [==============================] - 0s 6ms/step - loss: 0.9898 - accuracy: 0.5203
Epoch 940/1000
37/37 [==============================] - 0s 6ms/step - loss: 0.9882 - accuracy: 0.5212
Epoch 941/1000
37/37 [==============================] - 0s 6ms/step - loss: 0.9907 - accuracy: 0.5217
Epoch 942/1000
37/37 [==============================] - 0s 6ms/step - loss: 0.9901 - accuracy: 0.5207
Epoch 943/1000
37/37 [==============================] - 0s 7ms/step - loss: 0.9887 - accuracy: 0.5241
Epoch 944/1000
37/37 [==============================] - 0s 8ms/step - loss: 0.9900 - accuracy: 0.5218
Epoch 945/1000
37/37 [==============================] - 0s 8ms/step - loss: 0.9897 - accuracy: 0.5193
Epoch 946/1000
37/37 [==============================] - 0s 6ms/step - loss: 0.9913 - accuracy: 0.5197
Epoch 947/1000
37/37 [==============================] - 0s 9ms/step - loss: 0.9901 - accuracy: 0.

In [42]:
def test_model(model, features):
    return model.predict(features)

In [43]:
def accuracy(y, pred):
    yt = np.squeeze(y.T)
    a = np.sum(yt==pred)/y.shape[0]
    return a

In [44]:
pred_dev = np.argmax(test_model(model,X_dev),axis=1)

53/53 [==============================] - 0s 1ms/step


In [45]:
acc_dev = accuracy(y_dev, pred_dev)
print('The accuracy for the dev set is: ' + str(acc_dev))

The accuracy for the dev set is: 0.5343195266272189


In [46]:
## use the dev set accuracy to tune hyperparameters

In [47]:
pred_test = np.argmax(test_model(model,X_test),axis=1)

52/52 [==============================] - 0s 1ms/step


In [48]:
acc_test = accuracy(y_test, pred_test)
print('The accuracy for the test set is: ' + str(acc_test))

The accuracy for the test set is: 0.49908480780964004
